# RECUPERATION ET Construction DATASET

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

# Charger le dataset HuggingFace
# dataset = load_dataset("https://huggingface.co/datasets/stanfordnlp/sentiment140/")

# df = pd.DataFrame(dataset['train'])
dataset = pd.read_csv(
    "/content/drive/MyDrive/Colab Notebooks/training.1600000.processed.noemoticon.csv",
    # sep=",",
    encoding='latin1',
    header=None
)

dataset.columns = ["sentiment", "ids", "date", "query", "user", "text"]
print(f"Shape du dataset : {dataset.shape}")
print("__ Infos générales __")
print(dataset.info())
print("\n__ Colonnes __")
print(dataset.columns.tolist())
dataset.head()

Shape du dataset : (1600000, 6)
__ Infos générales __
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   ids        1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   text       1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB
None

__ Colonnes __
['sentiment', 'ids', 'date', 'query', 'user', 'text']


,sentiment,ids,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:

print("___  Statistiques descriptives ___ \n")
print("\t _Valeurs numeriques ")
print(dataset.describe())
print("\t _Valeurs Categorielles ")
print(dataset.describe(include=['object']))
print("\n___ Distribution des sentiments ___")
print(dataset['sentiment'].value_counts())


___  Statistiques descriptives ___ 

	 _Valeurs numeriques 
          sentiment           ids
count  1.600000e+06  1.600000e+06
mean   2.000000e+00  1.998818e+09
std    2.000001e+00  1.935761e+08
min    0.000000e+00  1.467810e+09
25%    0.000000e+00  1.956916e+09
50%    2.000000e+00  2.002102e+09
75%    4.000000e+00  2.177059e+09
max    4.000000e+00  2.329206e+09
	 _Valeurs Categorielles 
                                date     query      user  \
count                        1600000   1600000   1600000   
unique                        774363         1    659775   
top     Mon Jun 15 12:53:14 PDT 2009  NO_QUERY  lost_dog   
freq                              20   1600000       549   

                             text  
count                     1600000  
unique                    1581466  
top     isPlayer Has Died! Sorry   
freq                          210  

___ Distribution des sentiments ___
sentiment
0    800000
4    800000
Name: count, dtype: int64


In [ ]:
print("___  Valeurs manquantes ___ \n")
missing = dataset.isnull().sum()
missing_percent = (missing / len(dataset)) * 100
missing_df = pd.DataFrame({
    'Colonne': missing.index,
    'Manquantes': missing.values,
    'Pourcentage': missing_percent.values
})
print(missing_df[missing_df['Manquantes'] > 0].sort_values('Manquantes', ascending=False))
missing_df.head()

___  Valeurs manquantes ___ 

Empty DataFrame
Columns: [Colonne, Manquantes, Pourcentage]
Index: []


,Colonne,Manquantes,Pourcentage
0,sentiment,0,0.0
1,ids,0,0.0
2,date,0,0.0
3,query,0,0.0
4,user,0,0.0


## ***Echantillonnage 1(400000)***: inferer genre et date




In [ ]:
# Remove rows where 'user' is 'lost_dog'
df_filtered = dataset[dataset['user'] != 'lost_dog']

# Determine sample size for each sentiment to get a total of 100000 balanced samples
sample_size_per_sentiment = 400000 // 2

# Sample from each sentiment, ensuring not to sample more than available rows
sampled_sentiment_0 = df_filtered[df_filtered['sentiment'] == 0].sample(n=min(len(df_filtered[df_filtered['sentiment'] == 0]), sample_size_per_sentiment), random_state=42)
sampled_sentiment_4 = df_filtered[df_filtered['sentiment'] == 4].sample(n=min(len(df_filtered[df_filtered['sentiment'] == 4]), sample_size_per_sentiment), random_state=42)

# Concatenate the sampled dataframes
sampled_df = pd.concat([sampled_sentiment_0, sampled_sentiment_4]).sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Taille : {sampled_df.shape}")
print("Distribution of sentiments:")
print(sampled_df['sentiment'].value_counts())
display(sampled_df.head())

Taille : (400000, 6)
Distribution of sentiments:
sentiment
0    200000
4    200000
Name: count, dtype: int64


,sentiment,ids,date,query,user,text
0,0,2326119192,Thu Jun 25 06:34:47 PDT 2009,NO_QUERY,AmandaDz22,Good morning! Hitting up the gym to help start...
1,0,2175399116,Mon Jun 15 00:52:50 PDT 2009,NO_QUERY,mini_ritz,@Ddubs_Ky_Monkey Id hate it if it was the end
2,0,1678662916,Sat May 02 07:22:59 PDT 2009,NO_QUERY,Marxk,bad day.....damn you @311. What kind of tic...
3,0,2233022242,Thu Jun 18 20:43:25 PDT 2009,NO_QUERY,courtknee18u2,@honey_child I'm 5'1Â½&quot;.
4,4,1979390123,Sun May 31 02:50:28 PDT 2009,NO_QUERY,melojean,"Susan Boyle(Bwaakk), Google Wave, BGT, Diversi..."


###Fonction pour inferer genre et date

In [ ]:
from dateutil import parser
# --------------__________Fonction pour séparer date et heure
def split_date_time(date_str):
    try:
        dt = parser.parse(date_str)
        return dt.strftime("%Y-%m-%d"), dt.strftime("%H:%M:%S")
    except Exception as e:
        print(f"Erreur pour {date_str}: {e}")
        return None, None

# Appliquer à la colonne 'date'
sampled_df[["tweet_date", "tweet_time"]] = sampled_df["date"].apply(
    lambda x: pd.Series(split_date_time(x))
)

# sampled_df.head()

In [ ]:
###-----------------------____________Fonction pour enrichissement genre
!pip install gender_guesser
!pip install names-dataset

from gender_guesser.detector import Detector as GenderGuesserDetector
from names_dataset import NameDataset,NameWrapper
import re

def extract_first_name(username):
    """Extrait le prénom d'un username."""
    if not username:
        return None

    # Supprimer les underscores, caractères spéciaux initiaux et chiffres
    clean_name = re.sub(r'^[_@]+', '', username).strip()
    clean_name = re.sub(r'\d+', '', clean_name)  # Supprimer tous les chiffres

    if not clean_name:
        return None

    # Extraire le premier mot (en tenant compte des majuscules)
    words = re.findall(r'[A-Z][a-z]+', clean_name)
    if words:
        return words[0]
    else:
        # Si aucun mot avec majuscule, essayer de récupérer la première partie significative
        # en divisant par les underscores, puis par les espaces.
        parts_by_underscore = [p for p in clean_name.split('_') if p.strip()]
        if parts_by_underscore:
            first_part = parts_by_underscore[0]
            words_by_space = [w for w in first_part.split() if w.strip()]
            if words_by_space:
                return words_by_space[0].capitalize()
        return None


# Cache pour éviter les appels redondants
gender_cache = {}

# Initialiser les détecteurs
gender_guesser_detector = GenderGuesserDetector(case_sensitive=False)
nd = NameDataset()
def extract_gender_from_names_dataset(name_info):
    """Extrait le genre d'un résultat names-dataset avec gestion des probabilités."""
    if not name_info:
        return None

    # Cas 1: C'est un dictionnaire
    if isinstance(name_info, dict):
        # Vérifier la clé 'gender' directe
        if 'gender' in name_info:
            gender = name_info['gender']
            if gender in ['M', 'Male', 'male', 'MALE']:
                return "male"
            elif gender in ['F', 'Female', 'female', 'FEMALE']:
                return "female"

        # Vérifier les probabilités si disponibles
        elif 'probability' in name_info:
            prob = name_info['probability']
            gender = name_info.get('gender')
            # Seuil de confiance: accepter si probabilité > 0.5
            if prob and prob > 0.5:
                if gender in ['M', 'Male', 'male', 'MALE']:
                    return "male"
                elif gender in ['F', 'Female', 'female', 'FEMALE']:
                    return "female"

        # Vérifier dans les sous-clés
        for key in ['first_name', 'name', 'data']:
            if key in name_info:
                sub_result = extract_gender_from_names_dataset(name_info[key])
                if sub_result:
                    return sub_result

    # Cas 2: C'est un objet avec attributs
    elif hasattr(name_info, 'gender'):
        gender = name_info.gender
        # Vérifier aussi la probabilité si elle existe
        if hasattr(name_info, 'probability'):
            prob = name_info.probability
            if prob and prob > 0.5:
                if gender in ['M', 'Male', 'male', 'MALE']:
                    return "male"
                elif gender in ['F', 'Female', 'female', 'FEMALE']:
                    return "female"
        else:
            # Pas de probabilité, on fait confiance au résultat
            if gender in ['M', 'Male', 'male', 'MALE']:
                return "male"
            elif gender in ['F', 'Female', 'female', 'FEMALE']:
                return "female"

    # Cas 3: C'est une liste (plusieurs résultats)
    elif isinstance(name_info, list) and len(name_info) > 0:
        # Prendre le premier résultat ou celui avec la meilleure probabilité
        best_result = None
        best_prob = 0

        for item in name_info:
            result = extract_gender_from_names_dataset(item)
            if result:
                # Si on a des probabilités, choisir le meilleur
                if isinstance(item, dict) and 'probability' in item:
                    if item['probability'] > best_prob:
                        best_prob = item['probability']
                        best_result = result
                elif not best_result:
                    best_result = result

        return best_result

    return None

def infer_gender_fallback(name):
    """Détermine le genre d'un prénom en utilisant plusieurs sources."""
    if not name or not isinstance(name, str):
        return "unknown"

    # Vérifier le cache
    if name in gender_cache:
        return gender_cache[name]

    # 1. Essayer avec gender_guesser (plus complet, prioritaire)
    try:
        gender_guesser_result = gender_guesser_detector.get_gender(name)
        if gender_guesser_result not in ["unknown", "andy"]:
            if gender_guesser_result == "mostly_male":
                gender_cache[name] = "male"
                return "male"
            elif gender_guesser_result == "mostly_female":
                gender_cache[name] = "female"
                return "female"
            elif gender_guesser_result in ["male", "female"]:
                gender_cache[name] = gender_guesser_result
                return gender_guesser_result
    except Exception as e:
        print(f"Erreur avec gender-guesser pour {name}: {e}")


    # 2. Essayer avec names-dataset - Méthode search()
    try:
        search_result = nd.search(name)
        gender = extract_gender_from_names_dataset(search_result)
        if gender:
            gender_cache[name] = gender
            return gender
    except Exception as e:
        pass

    # 3. Essayer avec NameWrapper
    try:
        name_wrapper = NameWrapper(nd.first_names)
        name_info = name_wrapper.search(name)
        gender = extract_gender_from_names_dataset(name_info)
        if gender:
            gender_cache[name] = gender
            return gender
    except Exception as e:
        pass

    # Si aucune bibliothèque ne trouve le genre
    gender_cache[name] = "unknown"
    return "unknown"


# Appliquer aux données
sampled_df["user_gender"] = sampled_df["user"].apply(extract_first_name).apply(
    lambda x: infer_gender_fallback(x) if x else "unknown"
)

# Afficher quelques statistiques
print(f"Taille : {sampled_df.shape}")
print("\nDistribution des genres:")
print(sampled_df["user_gender"].value_counts())
# print(f"\nNombre de prénoms en cache: {len(gender_cache)}")
print("Distribution of sentiments:")
print(sampled_df['sentiment'].value_counts())
display(sampled_df.head())

Taille : (400000, 9)

Distribution des genres:
user_gender
unknown    331498
female      44766
male        23736
Name: count, dtype: int64
Distribution of sentiments:
sentiment
0    200000
4    200000
Name: count, dtype: int64


,sentiment,ids,date,query,user,text,tweet_date,tweet_time,user_gender
0,0,2326119192,Thu Jun 25 06:34:47 PDT 2009,NO_QUERY,AmandaDz22,Good morning! Hitting up the gym to help start...,2009-06-25,06:34:47,female
1,0,2175399116,Mon Jun 15 00:52:50 PDT 2009,NO_QUERY,mini_ritz,@Ddubs_Ky_Monkey Id hate it if it was the end,2009-06-15,00:52:50,unknown
2,0,1678662916,Sat May 02 07:22:59 PDT 2009,NO_QUERY,Marxk,bad day.....damn you @311. What kind of tic...,2009-05-02,07:22:59,unknown
3,0,2233022242,Thu Jun 18 20:43:25 PDT 2009,NO_QUERY,courtknee18u2,@honey_child I'm 5'1Â½&quot;.,2009-06-18,20:43:25,unknown
4,4,1979390123,Sun May 31 02:50:28 PDT 2009,NO_QUERY,melojean,"Susan Boyle(Bwaakk), Google Wave, BGT, Diversi...",2009-05-31,02:50:28,unknown


##***Echatillonnage 2 et Generation rest Attribut***

###Echantillonnage

In [ ]:
##-------- Suprression des genres non identifier
df_filtered2 = sampled_df[sampled_df['user_gender'] != 'unknown']

# Determine sample size for each sentiment to get a total of 100000 balanced samples
sample_size_per_sentiment = 60000 // 2

# Sample from each sentiment, ensuring not to sample more than available rows
sampled_sentiment_0 = df_filtered2[df_filtered2['sentiment'] == 0].sample(n=min(len(df_filtered2[df_filtered2['sentiment'] == 0]), sample_size_per_sentiment), random_state=42)
sampled_sentiment_4 = df_filtered2[df_filtered2['sentiment'] == 4].sample(n=min(len(df_filtered2[df_filtered2['sentiment'] == 4]), sample_size_per_sentiment), random_state=42)
# Concatenate the sampled dataframes
sampled_df2 = pd.concat([sampled_sentiment_0, sampled_sentiment_4]).sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Taille : {sampled_df2.shape}")
print(sampled_df2['user_gender'].value_counts())
print(sampled_df2['sentiment'].value_counts())

Taille : (60000, 9)
user_gender
female    39257
male      20743
Name: count, dtype: int64
sentiment
0    30000
4    30000
Name: count, dtype: int64


In [ ]:
### ------------Equilibrage genre

# Séparer le DataFrame par genre
df_female = sampled_df2[sampled_df2['user_gender'] == 'female']
df_male = sampled_df2[sampled_df2['user_gender'] == 'male']

# Calculer le nombre de lignes 'female' à conserver
rows_to_keep_female = len(df_female) - 15000

# S'assurer que le nombre de lignes à conserver n'est pas négatif
if rows_to_keep_female < 0:
    rows_to_keep_female = 0
    print("Warning: Not enough 'female' rows to remove 7000. All 'female' rows will be kept.")

# Échantillonner les lignes 'female' restantes
sampled_df_female = df_female.sample(n=rows_to_keep_female, random_state=42)

# Concaténer le DataFrame 'female' échantillonné avec le DataFrame 'male'
sampled_df2_updated = pd.concat([sampled_df_female, df_male]).sample(frac=1, random_state=42).reset_index(drop=True)

# Sauvegarder le résultat
sampled_df2_updated.to_csv("/content/drive/MyDrive/Colab Notebooks/sentiment140_enriched_with_date_genre.csv", index=False)

print(f"Taille : {sampled_df2_updated.shape}")
print(sampled_df2_updated['user_gender'].value_counts())
print(sampled_df2_updated['sentiment'].value_counts())
display(sampled_df2_updated.head())

Taille : (45000, 9)
user_gender
female    24257
male      20743
Name: count, dtype: int64
sentiment
0    22549
4    22451
Name: count, dtype: int64


,sentiment,ids,date,query,user,text,tweet_date,tweet_time,user_gender
0,0,2198368314,Tue Jun 16 15:58:58 PDT 2009,NO_QUERY,sean_okeefe,"Thanks MASN, no Yankee game due to local black...",2009-06-16,15:58:58,male
1,0,1548880457,Fri Apr 17 22:16:12 PDT 2009,NO_QUERY,eboni_ife,@gylliwilli get em Gilly!!!! see yall in a min...,2009-04-17,22:16:12,female
2,0,2296307442,Tue Jun 23 09:01:26 PDT 2009,NO_QUERY,WayneR79,Ah meee's not well Meeee's needs some love ...,2009-06-23,09:01:26,male
3,4,1677763816,Sat May 02 03:51:31 PDT 2009,NO_QUERY,laura_chandler,is home.,2009-05-02,03:51:31,female
4,0,2013541755,Tue Jun 02 22:56:04 PDT 2009,NO_QUERY,Danial79,Just been watching some E3 videos on Gamespot....,2009-06-02,22:56:04,male


###Génération des 4 attributs


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import timedelta
import re

def _load_user_profiles():
    """
    Profils d'utilisateurs basés sur Cha et al. (2010)
    """
    return {
        'influencer': {  # 1% des users, ~77% des followers
            'followers_range': (5000, 100000),
            'following_range': (500, 5000),
            'posts_per_day': (5, 20),
            'engagement_multiplier': 5.0,
            'probability': 0.01
        },
        'active': {  # 9% des users, très actifs
            'followers_range': (500, 5000),
            'following_range': (300, 3000),
            'posts_per_day': (3, 15),
            'engagement_multiplier': 2.0,
            'probability': 0.09
        },
        'regular': {  # 40% des users, usage normal
            'followers_range': (50, 500),
            'following_range': (100, 1000),
            'posts_per_day': (1, 5),
            'engagement_multiplier': 1.0,
            'probability': 0.40
        },
        'casual': {  # 50% des users, peu actifs
            'followers_range': (5, 50),
            'following_range': (20, 200),
            'posts_per_day': (0.1, 2),
            'engagement_multiplier': 0.3,
            'probability': 0.50
        }
    }

def _detect_user_profile(username, text, sentiment): # Removed 'self'
    """
    Détecte le profil d'utilisateur basé sur des indices
    """
    text_lower = text.lower()

    # Indices d'influenceur
    influencer_keywords = ['rt @', 'follow', 'check out', 'subscribe', 'link', 'http']
    is_likely_influencer = sum(1 for kw in influencer_keywords if kw in text_lower) >= 2

    # Username court et simple = compte ancien/influent (heuristique)
    is_short_username = len(username) < 8 and not any(c.isdigit() for c in username)

    # Déterminer le profil
    if is_likely_influencer or is_short_username:
        profiles = ['influencer', 'active', 'regular']
        probs = [0.3, 0.5, 0.2]
    else:
        profiles = ['regular', 'casual']
        probs = [0.4, 0.6]

    return np.random.choice(profiles, p=probs)

def generate_followers(user_profiles_dict, username, account_age_days, profile_type, text): # Removed 'self', added 'user_profiles_dict'
    profile = user_profiles_dict[profile_type]
    min_followers, max_followers = profile['followers_range']

    # Base : power law
    alpha = 2.276  # Exposant observé par Kwak et al.
    followers_base = np.random.pareto(alpha) * 50 + min_followers

    # Ajustement 1 : Ancienneté (plus vieux = plus de followers)
    age_factor = np.log1p(account_age_days) / 10
    followers = followers_base * (1 + age_factor)

    # Ajustement 2 : Indices dans le texte
    text_lower = text.lower()

    # URLs/links = plus de followers (content creators)
    if 'http' in text_lower or 'www' in text_lower:
        followers *= 1.5

    # Mentions multiples = networker
    mentions_count = text_lower.count('@')
    if mentions_count >= 2:
        followers *= 1.3

    # Hashtags = engagement
    hashtags_count = text_lower.count('#')
    if hashtags_count >= 2:
        followers *= 1.2

    # Limiter aux ranges réalistes
    followers = np.clip(followers, min_followers, max_followers)

    return int(followers)

def generate_posts(user_profiles_dict, account_age_days, profile_type, followers): # Removed 'self', added 'user_profiles_dict'
    profile = user_profiles_dict[profile_type]
    min_rate, max_rate = profile['posts_per_day']

    # Taux de tweets/jour basé sur le profil
    daily_rate = np.random.uniform(min_rate, max_rate)

    # Ajustement : corrélation avec followers
    follower_boost = np.log1p(followers) / 100
    daily_rate *= (1 + follower_boost)

    # Nombre total de posts
    total_posts = int(account_age_days * daily_rate)

    # Ajouter du bruit réaliste (burst posting)
    noise = np.random.gamma(2, 0.5)
    total_posts = int(total_posts * noise)

    return max(10, min(50000, total_posts))

def generate_likes(user_profiles_dict, text, sentiment, followers, profile_type): # Removed 'self', added 'user_profiles_dict'
    profile = user_profiles_dict[profile_type]
    engagement_mult = profile['engagement_multiplier']

    # Base : log-normal distribution
    mu = 0.5  # médiane ~1.6 likes
    sigma = 2.0  # forte variance
    base_likes = np.random.lognormal(mu, sigma)

    # Facteur 1 : Influence (followers)
    # Formule basée sur Cha et al.: likes ≈ 0.01 * followers^0.8
    influence_factor = (followers ** 0.8) * 0.01

    # Facteur 2 : Sentiment (TRÈS IMPORTANT)
    if sentiment == 0:  # Négatif
        sentiment_boost = 2.2  # +120% engagement pour les plaintes
    else:  # Positif
        sentiment_boost = 1.0

    # Facteur 3 : Contenu du tweet
    text_lower = text.lower()
    content_boost = 1.0

    # Questions génèrent plus d'engagement
    if '?' in text:
        content_boost *= 1.4

    # Mentions = conversations = plus d'engagement
    mentions_count = text_lower.count('@')
    content_boost *= (1 + mentions_count * 0.2)

    # Hashtags populaires
    hashtags_count = text_lower.count('#')
    content_boost *= (1 + hashtags_count * 0.15)

    # URLs = contenu partageable
    if 'http' in text_lower:
        content_boost *= 1.3

     # Mots émotionnels forts
    emotion_words = [
    # Émotions positives fortes
    'love', 'amazing', 'best', 'awesome', 'perfect', 'excellent', 'fantastic',
    'wonderful', 'brilliant', 'outstanding', 'incredible', 'magnificent',
    'superb', 'spectacular', 'exceptional', 'phenomenal', 'marvelous',
    'fabulous', 'delightful', 'terrific', 'splendid', 'gorgeous', 'beautiful',
    'thrilled', 'ecstatic', 'joyful', 'passionate', 'adore', 'treasure',

    # Émotions négatives fortes
    'hate', 'terrible', 'worst', 'horrible', 'awful', 'disgusting', 'pathetic',
    'dreadful', 'atrocious', 'abysmal', 'appalling', 'horrendous', 'deplorable',
    'miserable', 'wretched', 'despicable', 'worthless', 'useless', 'hideous',
    'revolting', 'repulsive', 'nasty', 'vile', 'detestable', 'loathe',
    'furious', 'enraged', 'outraged', 'devastated', 'heartbroken', 'despair']
    emotion_count = sum(1 for word in emotion_words if word in text_lower)
    content_boost *= (1 + emotion_count * 0.25)

    # Calculer les likes finaux
    likes = base_likes * influence_factor * sentiment_boost * content_boost * engagement_mult

    # Distribution réaliste : beaucoup de tweets à 0-5 likes, peu à 100+
    if np.random.random() < 0.7:  # 70% des tweets ont peu de likes
        likes *= np.random.uniform(0.1, 0.5)

    return max(0, int(likes))

def generate_location(text, sentiment, username): # Removed 'self'

    text_lower = text.lower()

    # Détection directe dans le texte (très précis)
    location_keywords = {
        'New York, NY': ['nyc', 'new york', 'manhattan', 'brooklyn', 'bronx', 'queens'],
        'Los Angeles, CA': ['la', 'los angeles', 'hollywood', 'socal', 'beverly'],
        'Chicago, IL': ['chicago', 'chi-town', 'windy city'],
        'San Francisco, CA': ['sf', 'san francisco', 'bay area', 'silicon valley'],
        'Houston, TX': ['houston', 'htx'],
        'Phoenix, AZ': ['phoenix', 'arizona', 'phx'],
        'Philadelphia, PA': ['philly', 'philadelphia'],
        'San Antonio, TX': ['san antonio'],
        'San Diego, CA': ['san diego'],
        'Dallas, TX': ['dallas', 'dfw'],
        'Austin, TX': ['austin', 'atx'],
        'Seattle, WA': ['seattle'],
        'Boston, MA': ['boston'],
        'London, UK': ['london', 'uk', 'britain', 'england'],
        'Toronto, Canada': ['toronto', 'canada'],
        'Sydney, Australia': ['sydney', 'australia']
    }

    # Chercher dans le texte
    for location, keywords in location_keywords.items():
        for keyword in keywords:
            if keyword in text_lower:
                return location

    # Fallback : distribution réaliste basée sur la population Twitter 2009
    # USA dominait Twitter en 2009 (~60%)
    locations = [
        'New York, NY', 'Los Angeles, CA', 'Chicago, IL', 'San Francisco, CA',
        'Houston, TX', 'Phoenix, AZ', 'Philadelphia, PA', 'Dallas, TX',
        'San Diego, CA', 'Austin, TX', 'Seattle, WA', 'Boston, MA',
        'London, UK', 'Toronto, Canada', 'Unknown'
    ]

    # Distribution basée sur l'usage Twitter 2009
    probs = [
        0.12,  # NY (hub majeur)
        0.10,  # LA
        0.08,  # Chicago
        0.09,  # SF (tech hub)
        0.05,  # Houston
        0.03,  # Phoenix
        0.04,  # Philadelphia
        0.04,  # Dallas
        0.04,  # San Diego
        0.03,  # Austin
        0.04,  # Seattle
        0.03,  # Boston
        0.08,  # London (UK était #2 après USA)
        0.03,  # Toronto
        0.20   # Unknown
    ]

    return np.random.choice(locations, p=probs)


####Fonction Principale enrichissement

In [ ]:
def enrich_with_ultra_realistic_attributes(df):
  user_profiles_dict = _load_user_profiles() # Initialize user_profiles
  enriched_rows = []

  for idx, row in df.iterrows():
      if idx % 10000 == 0:
          print(f" Progression: {idx:,}/{len(df):,} tweets ({idx/len(df)*100:.1f}%)")

      username = row["user"]
      text = row["text"]
      sentiment = row["sentiment"]
      tweet_date = pd.to_datetime(row["tweet_date"])

      # Seed pour cohérence
      seed = hash(username) % (2**32)
      np.random.seed(seed)

      # Calculer l'âge du compte (Attribut 1)
      days_into_2009 = max(1, (tweet_date - pd.Timestamp('2009-01-01')).days)
      max_age = min(1095, 365 + days_into_2009)
      account_age_days = int(np.random.lognormal(5.5, 1.2)) % max_age + 30

      # Détecter le profil utilisateur
      profile_type = _detect_user_profile(username, text, sentiment) # Correct call


      followers = generate_followers(user_profiles_dict, username, account_age_days, profile_type, text) # Correct call

      # ATTRIBUT 4 : Posts (IMPORTANT)
      num_posts = generate_posts(user_profiles_dict, account_age_days, profile_type, followers) # Correct call

      # ATTRIBUT 10 : Likes (CRITIQUE)
      likes = generate_likes(user_profiles_dict, text, sentiment, followers, profile_type) # Correct call

      # ATTRIBUT 3 : Location (IMPORTANT)
      location = generate_location(text, sentiment, username) # Correct call

      # Attribut 5 : Number of retweets par l'utilisateur (IMPORTANT pour influence)

      # Calculer les retweets basé sur le profil et l'activité

      profile = user_profiles_dict[profile_type]

      # Heuristique : ~15-30% des posts sont des retweets (observé sur Twitter 2009)
      if profile_type == 'influencer':
          retweet_ratio = np.random.uniform(0.10, 0.20)  # Influencers créent plus de contenu original
      elif profile_type == 'active':
          retweet_ratio = np.random.uniform(0.20, 0.30)  # Users actifs retweetent plus
      elif profile_type == 'regular':
          retweet_ratio = np.random.uniform(0.25, 0.35)
      else:  # casual
          retweet_ratio = np.random.uniform(0.30, 0.50)  # Casual users retweetent proportionnellement plus

      # Nombre de retweets = ratio * nombre de posts
      user_num_retweets = int(num_posts * retweet_ratio)

      # Ajustement : utilisateurs avec plus de followers retweetent plus (exposition)
      follower_boost = np.log1p(followers) / 100
      user_num_retweets = int(user_num_retweets * (1 + follower_boost))

      # Ajustement : détection de RT dans le texte actuel
      if text.startswith('RT @') or 'RT @' in text[:20]:
          # Ce tweet EST un retweet, booster légèrement le total de l'user
          user_num_retweets = int(user_num_retweets * 1.1)

      # Limiter aux valeurs réalistes
      user_num_retweets = max(0, min(user_num_retweets, int(num_posts * 0.6)))

      # Construire la ligne enrichie
      enriched_rows.append({
          # Original
          "text": text,
          "sentiment": sentiment,
          "user_account_age_days": account_age_days,      # Attribut 1
          "user_gender": row["user_gender"],               # Attribut 2
          "user_location": location,                       # Attribut 3
          "user_num_posts": num_posts,                     # Attribut 4
          "user_num_retweets": user_num_retweets,
          "user_num_followers": followers,                 # Attribut 6
          "tweet_hour": row["tweet_time"],                 # Attribut 7
          "tweet_num_likes": likes,                        # Attribut 8
          "user_profile_type": profile_type                # Added profile_type to the row
      })

  enriched_df = pd.DataFrame(enriched_rows)

  # Statistiques finales
  print("  GÉNÉRATION TERMINÉE")
  print(f"\n Total: {len(enriched_df):,} tweets enrichis")

  print("\n Profils d'utilisateurs détectés:")
  for profile, count in enriched_df['user_profile_type'].value_counts().items(): # This column is now available
      print(f"   {profile:12s}: {count:,} ({count/len(enriched_df)*100:.1f}%)")

  return enriched_df

### Enrichissement echantillons

In [ ]:
# Charger le dataset
print(" Chargement du dataset...")
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sentiment140_enriched_with_date_genre.csv")
print(f"   {len(df):,} tweets chargés\n")

# Enrichir
enriched_df = enrich_with_ultra_realistic_attributes(df)

# Afficher quelques statistiques
print(f"Taille : {enriched_df.shape}")
print("\nDistribution des genres:")
# print(sampled_df["user_gender"].value_counts())
# # print(f"\nNombre de prénoms en cache: {len(gender_cache)}")
print("Distribution of sentiments:")
print(enriched_df['sentiment'].value_counts())
display(enriched_df.head())

# Sauvegarder
output_file = "/content/drive/MyDrive/Colab Notebooks/sentiment140_all_attributes.csv"
enriched_df.to_csv(output_file, index=False)


 Chargement du dataset...
   45,000 tweets chargés

 Progression: 0/45,000 tweets (0.0%)
 Progression: 10,000/45,000 tweets (22.2%)
 Progression: 20,000/45,000 tweets (44.4%)
 Progression: 30,000/45,000 tweets (66.7%)
 Progression: 40,000/45,000 tweets (88.9%)
  GÉNÉRATION TERMINÉE

 Total: 45,000 tweets enrichis

 Profils d'utilisateurs détectés:
   casual      : 24,650 (54.8%)
   regular     : 17,213 (38.3%)
   active      : 1,947 (4.3%)
   influencer  : 1,190 (2.6%)
Taille : (45000, 11)

Distribution des genres:
Distribution of sentiments:
sentiment
0    22549
4    22451
Name: count, dtype: int64


,text,sentiment,user_account_age_days,user_gender,user_location,user_num_posts,user_num_retweets,user_num_followers,tweet_hour,tweet_num_likes,user_profile_type
0,"Thanks MASN, no Yankee game due to local black...",0,191,male,"Los Angeles, CA",595,180,186,15:58:58,0,regular
1,@gylliwilli get em Gilly!!!! see yall in a min...,0,444,female,"Los Angeles, CA",597,198,13,22:16:12,1,casual
2,Ah meee's not well Meeee's needs some love ...,0,415,male,Unknown,3304,978,90,09:01:26,0,regular
3,is home.,4,406,female,"New York, NY",217,86,50,03:51:31,0,casual
4,Just been watching some E3 videos on Gamespot....,0,246,male,"Toronto, Canada",390,127,88,22:56:04,2,regular
